In [1]:
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd

## First part - get Data: Basic functions that will be used 
Functions include:
getPagesToCrawl - a function that creates a list of pages that should be used in order to achieve the full data.
getTableDataFromPage - from a single page we get back a table that stores the data
cleanHtmlCode - the first step that we take in order to clean our data, we remove the HTML from it and leave the info.
createDf - gets 8 lists that contains data and return a df

In [2]:
def getPagesToCrawl():
    baseUrl="https://www.start.umd.edu/gtd/search/Results.aspx?page="
    amountPages = "&count=2000" 
    pageList = []
    
    for currentPageNumber in range(1,102):
        pageList.append(baseUrl + str(currentPageNumber) +amountPages)
        
    return pageList

In [3]:
def getTableDataFromPage(pageUrl):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response = requests.get(pageUrl,headers=user_agent)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table')
    return table

In [4]:
def cleanHtmlCode(tableData):
    GTD_ID = []
    DATE = []
    COUNTRY = []
    CITY = [] 
    PERPETRATOR_GROUP = []
    FATALITIES = []
    INJURED = []
    TARGET_TYPE = []

    listRows = []
    for tr in tableData.findAll("tr",attrs={}):
        for td in tr:
            if(td != "\n"):
                listRows.append(td.text)


    # A lot of data that we don't need (The name of the columns (8)) 
    for i in range(8):
        listRows.pop(0)
    
    for i in range(len(listRows)):
        iMod = i%8
        if(iMod == 0):
            GTD_ID.append(listRows[i])
        elif(iMod == 1):
            DATE.append(listRows[i])
        elif(iMod == 2):
            COUNTRY.append(listRows[i])
        elif(iMod == 3):
            CITY.append(listRows[i])
        elif(iMod == 4):
            PERPETRATOR_GROUP.append(listRows[i])
        elif(iMod == 5):
            FATALITIES.append(listRows[i])
        elif(iMod == 6):
            INJURED.append(listRows[i])
        elif(iMod == 7):
            TARGET_TYPE.append(listRows[i])
       
    
    return GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE


In [5]:
def createDf(GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE):
    df = pd.DataFrame({"GTD_ID":GTD_ID,
                   "DATE":DATE,
                   "COUNTRY":COUNTRY,
                   "CITY":CITY,
                   "PERPETRATOR_GROUP":PERPETRATOR_GROUP,
                   "FATALITIES":FATALITIES
                   ,"INJURED":INJURED,
                   "TARGET_TYPE":TARGET_TYPE,
                   })
    return df

## First part - get Data: Flow

In [7]:
# 8 Lists that will store the data.
GTD_ID = []
DATE = []
COUNTRY = []
CITY = [] 
PERPETRATOR_GROUP = []
FATALITIES = []
INJURED = []
TARGET_TYPE = []

In [11]:
# First we Need the URLS which will be used by the crawlling method.
pageList = getPagesToCrawl()

# For each page inside our pageList:
for page in pageList:
    
    # Get the table data from the page with getTableDataFromPage function.
    tableData = getTableDataFromPage(page)
    
    # Extract 8 lists (according to parameter) with cleanHtmlCode function.
    GTD_ID_temp,DATE_temp,COUNTRY_temp,CITY_temp,PERPETRATOR_GROUP_temp,FATALITIES_temp,INJURED_temp,TARGET_TYPE_temp = cleanHtmlCode(tableData)
    
    # The the temp lists and EXTEND the data to non-temp list.
    GTD_ID.extend(GTD_ID_temp)
    DATE.extend(DATE_temp)
    COUNTRY.extend(COUNTRY_temp)
    CITY.extend(CITY_temp)
    PERPETRATOR_GROUP.extend(PERPETRATOR_GROUP_temp)
    FATALITIES.extend(FATALITIES_temp)
    INJURED.extend(INJURED_temp)
    TARGET_TYPE.extend(TARGET_TYPE_temp)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [22]:
# After the crawlling method is done all we have left to do is create a DF with createDf function and save the df to csv file.
df = createDf(GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE) 
df.to_csv('GTD_Data_Frame.csv')

In [27]:
# Chceking some stuff part 1
df[df.DATE == "2001-09-11"]

,GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE
228045,200109110008,2001-09-11,Armenia,Yerevan,Unknown,1,0,Government (General)
228046,200109110007,2001-09-11,United States,Shanksville,Al-Qaida,44,9,"Private Citizens & Property,Government (Genera..."
228047,200109110006,2001-09-11,United States,Arlington,Al-Qaida,190,106,"Government (General),Military,Airports and Air..."
228048,200109110005,2001-09-11,United States,New York City,Al-Qaida,1385,10878,"Private Citizens & Property,Business,Airports ..."
228049,200109110004,2001-09-11,United States,New York City,Al-Qaida,1385,10878,"Private Citizens & Property,Business,Airports ..."
228050,200109110003,2001-09-11,Israel,Magal,Unknown,2,1,Police
228051,200109110002,2001-09-11,Democratic Republic of the Congo,Muhanga,Unknown,4,0,Private Citizens & Property
228052,200109110001,2001-09-11,Burundi,Gihange,Unknown,1,Unknown,"Transportation,Military"


In [28]:
# Chceking some stuff part 2
pd.read_csv('GTD_Data_Frame.csv', header=None)

C:\Users\lahav\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,0,1,2,3,4,5,6,7,8
0,NaN,GTD_ID,DATE,COUNTRY,CITY,PERPETRATOR_GROUP,FATALITIES,INJURED,TARGET_TYPE
1,0.0,201912310033,2019-12-31,China,Hong Kong,Unknown,0,0,Government (General)
2,1.0,201912310032,2019-12-31,India,Bagiot Dora,Unknown,0,1,Private Citizens & Property
3,2.0,201912310031,2019-12-31,Sudan,El Geneina,Unknown,2,0,"Government (General),Police"
4,3.0,201912310030,2019-12-31,Sudan,El Geneina,Unknown,2,1,Police
...,...,...,...,...,...,...,...,...,...
301179,301178.0,197001000003,1970-01-00,Japan,Fukouka,Unknown,Unknown,Unknown,Government (Diplomatic)
301180,301179.0,197001000002,1970-01-00,Greece,Athens,Unknown,Unknown,Unknown,Government (Diplomatic)
301181,301180.0,197001000001,1970-01-00,Philippines,Unknown,Unknown,1,0,Journalists & Media
301182,301181.0,197000000002,1970-00-00,Mexico,Mexico city,23rd of September Communist League,0,0,Government (Diplomatic)
